<a href="https://colab.research.google.com/github/valmirf/redes_neurais_esp/blob/main/PyTorch/Convolutional_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Análise de Sentimentos com Redes Convolucionais
Este notebook é adaptado do excelente material de Ben Trevett, disponível [aqui](https://github.com/bentrevett/pytorch-sentiment-analysis).

Tradicionalmente, CNNS são utilizadas pra análise de imagens devido a operação de filtragem e extração de características das camadas convolucionais. 

Então, porque usar CNNS em textos? Da mesma forma que filtros de tamanho quadrados (Ex. `3x3`) extraem características de uma imagem, filtros de uma dimensão (Ex. `1x2`), pode olhar pra duas palavras no texto (Bi-gram). Nos modelos de CNN, filtros de diferentes tamanhos (`1xn`), são n-grams no texto.

Nesta atividade, construiremos um modelo de aprendizado de máquina para detectar sentimentos (ou seja, detectar se uma frase é positiva ou negativa) usando PyTorch e TorchText. Isso será feito nas críticas de filmes, usando o [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/).

## Preparação dos Dados
Um dos principais conceitos do TorchText é o `Field` (campo). Eles definem como seus dados devem ser processados. Em nossa tarefa de classificação de sentimento, os dados consistem na avaliação da crítica e no sentimento sobre o filme, seja "pos" ou "neg".

Os parâmetros de um `Field` especificam como os dados devem ser processados.

Usamos o campo `TEXT` para definir como a revisão deve ser processada, e o campo` LABEL` para processar o sentimento.

No campo `TEXT` tem` tokenize = 'spacy'` como argumento. Isso define que a "tokenização" (o ato de dividir a string em "tokens" discretos) deve ser feita usando o tokenizer [spaCy] (https://spacy.io). Se nenhum argumento `tokenize` for passado, o padrão é simplesmente dividir a string em espaços.

`LABEL` é definido por um `LabelField`, um subconjunto especial da classe `Field` especificamente usado para lidar com rótulos. 

Para mais informações sobre `Fields`, acesse [aqui](https://github.com/pytorch/text/blob/master/torchtext/data/field.py).

As sementes aleatórias são definidas para reprodutibilidade.

In [ ]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', batch_first = True)
LABEL = data.LabelField(dtype = torch.float)

Outro recurso útil do TorchText é que ele oferece suporte para [conjuntos de dados](https://torchtext.readthedocs.io/en/latest/datasets.html) comuns usados ​​em processamento de linguagem natural (PNL).

Por exemplo, o Torchtext disponibiliza os conjuntos de dados de várias atividades de PNL:


* Análise de Sentimentos:
  * SST
  * IMDb
* Classificação de Perguntas:
  * TREC
  * Entailment
  * SNLI
  * MultiNLI
* Modelagem de Linguagem:
  * WikiText-2
  * WikiText103
  * PennTreebank
* Tradução:
  * Multi30k
  * IWSLT
  * WMT14
* _Tagging_ de Sequências:
  * UDPOS
  * CoNLL2000Chunking
* Resposta de Perguntas: 
  * BABI20


O código a seguir baixa automaticamente o conjunto de dados IMDb e divide-o em treino/teste utilizando `torchtext.datasets`. Ele processa os dados usando os `Campos` que foram definidos anteriormente. O conjunto de dados IMDb consiste em 50.000 críticas de filmes, cada uma marcada como sendo uma crítica positiva (`1`) ou negativa (`0`).

In [ ]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

Abaixo podemos ver a quantidade exemplos nos conjuntos de treino e no teste e um exemplo.

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(vars(train_data.examples[0]))

O conjunto de dados IMDb tem apenas divisões de treinamento/teste, portanto, se quisermos criar um conjunto de validação, temos que usar novamente o método `.split()`.

Por padrão, ele divide 70/30, no entanto, ao passar um argumento `split_ratio`, pode-se mudar a proporção da divisão, ou seja, um` split_ratio` de 0.8 significaria que 80% dos exemplos compõem o conjunto de validação e 20% compõem o conjunto de teste.

A semente aleatória no argumento `random_state`, é para garantir que obteremos a mesma divisão de validação/teste todas as vezes.

In [ ]:
#train_data, valid_data = train_data.split(random_state = random.seed(SEED))
valid_data, test_data = test_data.split(split_ratio=0.5, random_state = random.seed(SEED))

Abaixo podemos ver a quantidade exemplos nos conjuntos de treino, validação e no teste e um exemplo.

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Em seguida, iremos construir um _vocabulário_, que é uma tabela de busca, onde cada palavra única em seu conjunto de dados tem um _index_ correspondente (um identificador).

Cada _index_ é usado para construir um vetor _one-hot_ para cada palavra. Um vetor one-hot é um vetor em que todos os elementos são 0, exceto um, que é 1, e a dimensionalidade é o número total de palavras únicas no vocabulário, comumente denotado por $V$.

Considere a frase "The cat sat on the mat". O vocabulário (ou palavras únicas) nesta frase é (cat, mat, on, sat, the). Para representar cada palavra, será criado um vetor de zeros com comprimento igual ao vocabulário e, em seguida, colocaremos 1 no índice que corresponder à palavra. Essa abordagem é mostrada no diagrama a seguir.

![](https://github.com/valmirf/redes_neurais_esp/blob/main/PyTorch/FIG/one-hot.png?raw=true)

Porém, essa abordagem é ineficiente. Um vetor one-hot é escasso (ou seja, a maioria dos atributos é zero). Por exemplo, um vocabulário de 10.000, para codificar cada palavra, criaríamos um vetor em que 99,99% dos elementos são zero.

O número de palavras exclusivas neste conjunto de treinamento é superior a 100.000, o que significa que os vetores one-hot terão mais de 100.000 dimensões! Isso tornará o treinamento lento e possivelmente não caberá na GPU, além de um vetor com muitos zeros.

Existem duas maneiras de reduzir rapidamente o tamanho do vocabulário: pode-se apenas pegar as $n$ palavras mais comuns ou ignorar as palavras que aparecem menos de $m$ vezes. Faremos o primeiro, mantendo apenas as 25000 palavras principais.

O que acontece com as palavras que aparecem nos exemplos, mas são cortadas do vocabulário? Elas são substuídas por um token especial _unk_ ou ` ` (vazio). Por exemplo, se a frase for "Este filme é ótimo e eu adoro", mas a palavra "adoro" não estiver no vocabulário, ela se torna: "Este filme é ótimo e eu ` ` isso".

São adicionados dois especiais além dos que existem. Um é o token `<unk>` e o outro é um token `<pad>`.

Quando inserimos frases no modelo, a entrada é de um _lote_ delas por vez, ou seja, mais de uma de cada vez, e todas as frases do lote precisam ter o mesmo tamanho. Portanto, para garantir que cada frase do lote tenha o mesmo tamanho, qualquer frase menor do que a mais longa do lote é preenchida como mostra a figura abaixo.


![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment6.png?raw=1)

###Embeddings
Em seguida, vem o uso de embeddings de palavras pré-treinadas como opção aos _embeddings_ de palavras inicializados aleatoriamente. Esses vetores são obtidos simplesmente especificando quais vetores queremos e passando-os como um argumento para `build_vocab`. TorchText lida com o download dos vetores e os associa às palavras corretas em nosso vocabulário.

_Embeddings_ é uma estruturação de palavras numa representação "aprendida", em que palavras com significados semelhantes ou relacionadas têm uma codificação semelhante. A abordagem _embeddings_ para representar palavras e documentos pode ser considerada uma das principais descobertas do aprendizado profundo em problemas desafiadores de processamento de linguagem natural.

Nos _embeddings_ de palavras, as palavras individuais são representadas como vetores de valor real em um espaço vetorial pré-definido. Cada palavra é mapeada para um vetor e os valores do vetor são aprendidos por algum algoritmo de criação de _embeddings_, como por exemplo, os algoritmos Word2Vec e Glove.

![](https://github.com/valmirf/redes_neurais_esp/blob/main/PyTorch/FIG/embedding_example.png?raw=true)

Acima está um diagrama para um embedding de uma palavra. Cada palavra é representada como um vetor quadridimensional de valores de ponto flutuante. Outra maneira de pensar em um embedding é como "tabela de pesquisa". Depois que esses pesos foram aprendidos, podemos codificar cada palavra procurando o vetor denso a que corresponde na tabela. A Figura abaixo representa um mapa de Embeddings de críticas de filmes.

![](https://github.com/valmirf/redes_neurais_esp/blob/main/PyTorch/FIG/Embeddings.png?raw=true)


Aqui, usaremos os vetores "glove.6B.100d". Glove é o algoritmo usado para calcular os vetores, clique [aqui](https://nlp.stanford.edu/projects/glove/) para mais informações. 6B indica que esses vetores foram treinados em 6 bilhões de tokens e 100d indica que esses vetores são 100 -dimensional.

É possível encontrar outros vetores disponíveis [aqui](https://github.com/pytorch/text/blob/master/torchtext/vocab.py#L113).

A teoria é que esses vetores pré-treinados já têm palavras com significado semântico semelhante, próximas umas das outras no espaço vetorial, por exemplo, "agradável, "aprazível", "prazeroso", estão próximos. Isso dá à nossa camada de incorporação uma boa inicialização, pois ela não precisa aprender essas relações do zero.

In [ ]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

A etapa final da preparação dos dados é a criação dos iteradores. 

Usaremos um `BucketIterator` que é um tipo especial de iterador que retornará um lote de exemplos em que cada exemplo tem um comprimento semelhante, minimizando a quantidade de preenchimento por exemplo.

Para blocos de sequências preenchidas, todos os exemplos em um lote precisam ser classificados por seus comprimentos. Isso é tratado no iterador definindo `sort_within_batch = True`.



In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## Construção do Modelo

A primeira etapa é converter as palavras em _embeddings_ de palavras. É assim que se transforma palavras em 2 dimensões, onde cada palavra é colocada ao longo de um eixo e os elementos do vetor aprendido é outra dimensão. Considere a representação bidimensional da frase incorporada abaixo:

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment9.png?raw=1)

Pode-se usar um filtro $[n \times emb_{dim}]$. o filtro passará por $n$ palavras sequenciais inteiramente, já que sua largura será a dimensão `emb_dim`. Considere a imagem abaixo, com os vetores de palavras representados em verde. Temos 4 palavras com _embeddings_ de 5 ​​dimensões, criando um tensor de "imagem" [4x5]. O filtro [2x5], em amarelo, passará por duas palavras por vez (ou seja, bi-grams). A saída deste filtro (mostrado em vermelho) será um único número real que é a soma ponderada de todos os elementos cobertos pelo filtro.

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment12.png?raw=1)

A ideia é que o maior valor da saída calculada é a característica "mais importante" para determinar o sentimento da crítica. A rede definirá através dos cálculos de pesos pelo algoritmo _backpropagation_ qual as características mais importantes. 


### Implementação
As camadas convolucionais são implementadas com `nn.Conv2d`. O argumento `in_channels` é o número de "canais" de entrada da camada convolucional. O `out_channels` é o número de filtros e o `kernel_size` é o tamanho dos filtros. Cada um dos `kernel_size`s terá dimensão $[n \times emb\_dim]$ onde $n$ é o tamanho dos n-gramas e $emb_{dim}$ é a dimensão dos embeddings.

A segunda dimensão da entrada em uma camada `nn.Conv2d` deve ser a dimensão do canal. O tamanho da saída da camada convolucional depende do tamanho da entrada, e diferentes lotes contêm sentenças de diferentes comprimentos. Sem a camada de _max-pooling_, a entrada para nossa camada linear dependeria do tamanho da frase de entrada (não o que queremos). Uma opção para retificar isso seria cortar/preencher todas as sentenças com o mesmo comprimento; no entanto, com a camada de _max-pooling_, sempre sabemos que a entrada para a camada linear será o número total de filtros. 

**Nota**: há uma exceção se a frase forem mais curta do que o maior filtro usado. Pois, a frase terá que ser preenchida com o comprimento do maior filtro. Nos dados do IMDb não há comentários de apenas 5 palavras, então não precisamos nos preocupar com isso.

Neste modelo o token <pad> será ignorado. Isso ocorre porque queremos dizer explicitamente ao nosso modelo que os tokens de preenchimento <pad> são irrelevantes para determinar o sentimento de uma frase. Fazemos isso passando o índice do nosso token de pad como o argumento `padding_idx` para a camada `nn.Embedding`.

Pra utilizar uma lista de filtros, utiliza-se `nn.ModuleList`, uma função usada para manter uma lista de PyTorch `nn.Module`. Então, no método `forward`, iteramos através da lista aplicando cada camada convolucional para obter uma lista de saídas convolucionais, que também é alimentada por meio da _max-pooling_ em uma lista, antes de concatenar e passar pelas camadas _dropout_ e linear.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

###Parâmetros

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

Método para verificar o número de parâmetros do nosso modelo



In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

A seguir, os _embeddings_ pré-treinados são carregados


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

Em seguida, os pesos iniciais dos tokens desconhecidos e de preenchimento são inicializados com zero. 

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Treinamento do Modelo

São inicializados o otimizador, a função de perda (critério) e colocamos o modelo e o critério na GPU (se disponível)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

Função pra calcular a acurácia:

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

###Função de treinamento:

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

###Função de Teste:

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Função pra calcular o tempo de execução das épocas.

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Execução de treinamento.

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Aqui os resultados do teste é mostrado pelo código abaixo.

In [ ]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

## Entrada do usuário


A função `predict_sentiment` é modificada para aceitar um argumento de comprimento mínimo do tamanho do maior filtro. Se a sentença de entrada tokenizada for menor que tokens `min_len`, são acrescentados tokens de preenchimento (`<pad>`) para torná-la tokens de tamanho `min_len`.

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

Um exemplo de crítica negativa...

In [ ]:
predict_sentiment(model, "This film is terrible")

Um exemplo de crítica positiva...

In [ ]:
predict_sentiment(model, "This film is great")